In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from numpy import mean
from numpy import std
from matplotlib import pyplot
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.preprocessing import QuantileTransformer
from sklearn.pipeline import Pipeline

from keras.utils import np_utils

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [3]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
#npz files can only be loaded using /gdrive path
from google.colab import drive
drive.mount('/content/gdrive')
#%cd /gdrive

Mounted at /content/gdrive


In [5]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 20.4 MB/s eta 0:00:00


In [6]:
%matplotlib inline

import glob
import math
import ntpath
import random, os, sys
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
#import edfreader
#import models
import mne
#import pyeeg
import warnings
warnings.filterwarnings('ignore')

from matplotlib.backends.backend_pdf import PdfPages
from mne.datasets.sleep_physionet._utils import _fetch_one, _data_path, AGE_SLEEP_RECORDS, _check_subjects
from datetime import datetime
from mne import Epochs, pick_types, find_events
from mne.io import concatenate_raws, read_raw_edf
#from mne.time_frequency import psd_welch
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
import joblib

from sklearn.metrics import make_scorer, f1_score, accuracy_score, classification_report, log_loss
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_auc_score, roc_curve

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from xgboost import XGBClassifier
#import xgboost

In [7]:
VBS = True  # constant boolean to enable/disbale verbose
EPOCH_SEC_SIZE = 30  # Epoch duration selection
seed = 42  # seed value for the random seeds
batch_size = 64
number_of_files = 100 #100 npz files, 50 subjects, each suject has 2 nights, so 2 files per subject (50*2)=100

# values to label the stages
UNKNOWN = -1
W = 0
N1 = 1
N2 = 2
N3 = 3
REM = 4

# making string dictionary for the label values
label_dict = {
    "UNKNOWN"  : UNKNOWN,
    "W"        : W,
    "N1"       : N1,
    "N2"       : N2,
    "N3"       : N3,
    "REM"      : REM
}

# converting from label values to strings
class_dict = {
    -1: "UNKNOWN",
    0 : "W",
    1 : "N1",
    2 : "N2",
    3 : "N3",
    4 : "REM"
}

# annotation dictionary to convert from string to label values
annot2label = {
    "Sleep stage ?": -1,
    "Movement time": -1,
    "Sleep stage W": 0,
    "Sleep stage 1": 1,
    "Sleep stage 2": 2,
    "Sleep stage 3": 3,
    "Sleep stage 4": 3,
    "Sleep stage R": 4
}
#project_path = os.path.abspath(os.getcwd())  # finding the current project path in windows

In [8]:
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, MaxPool1D, Activation
from tensorflow.keras.layers import Reshape, LSTM, TimeDistributed, Bidirectional, BatchNormalization, Flatten, RepeatVector
from tensorflow.keras.optimizers import Adam
#from keras_contrib.layers import CRF

from scipy.signal import butter, lfilter


In [9]:
Fs = 100

In [ ]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/transfer_set_50_subs.csv',header=None)

In [10]:
transfer_df=pd.read_csv('/content/gdrive/My Drive/transfer_set_50_subs.csv',header=None)

In [11]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [ ]:
x_transfer_set=transfer_df.drop([3000],axis=1)

In [ ]:
x_transfer_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.026093,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.032245,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.004273,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.079087,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143


In [ ]:
y_transfer=transfer_df[3000]

In [ ]:
y_transfer

0         4
1         3
2         2
3         2
4         0
         ..
116713    2
116714    4
116715    4
116716    2
116717    2
Name: 3000, Length: 116718, dtype: int64

In [ ]:
x_transfer_set_rshp=np.array(x_transfer_set).reshape(116718,3000,1)

In [ ]:
x_transfer_set_rshp

array([[[ 0.0680144 ],
        [ 0.08291509],
        [ 0.07762775],
        ...,
        [-0.07714708],
        [-0.08483776],
        [-0.09012509]],

       [[ 0.1047289 ],
        [ 0.05451331],
        [-0.02013147],
        ...,
        [-0.17123062],
        [-0.14951577],
        [-0.12146743]],

       [[ 0.00228081],
        [-0.03130934],
        [-0.00767183],
        ...,
        [ 0.07277867],
        [ 0.05536155],
        [ 0.02965057]],

       ...,

       [[ 0.01339153],
        [ 0.03071675],
        [ 0.0298049 ],
        ...,
        [ 0.041659  ],
        [ 0.03299639],
        [ 0.02433378]],

       [[-0.0574574 ],
        [-0.06562874],
        [-0.10119811],
        ...,
        [-0.07091608],
        [-0.06851275],
        [-0.09014277]],

       [[-0.06574184],
        [-0.08269832],
        [-0.08636458],
        ...,
        [-0.15052423],
        [-0.08728115],
        [-0.04282768]]])

In [ ]:
y_transfer_=np_utils.to_categorical(y_transfer)

In [ ]:
y_transfer_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [ ]:
subject_481_tst_df=pd.read_csv('/content/gdrive/My Drive/physionet/ensemble/50_subs/x_test_sub.csv',header=None)

In [12]:
subject_481_tst_df=pd.read_csv('/content/gdrive/My Drive/sub_481/x_test_sub.csv',header=None)

In [13]:
subject_481_tst_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.026201,0.021701,0.037902,0.021701,0.022601,-0.004401,0.019901,0.011800,-0.037704,0.031151,...,0.068504,0.052753,0.062654,0.027101,-0.001251,-0.031404,0.004599,-0.030053,-0.027803,4
1,-0.102959,-0.104309,-0.099359,-0.129061,-0.121861,-0.101159,-0.045355,-0.039954,-0.031404,-0.017902,...,0.038802,0.087406,0.138260,0.129709,0.124309,0.086506,0.072105,0.023501,-0.022853,2
2,-0.122673,-0.129459,-0.115887,-0.080600,-0.076529,-0.056171,-0.057076,-0.059790,-0.055719,-0.036718,...,0.084071,0.064618,0.064166,0.050594,0.028427,0.027974,0.044713,0.058284,0.060999,2
3,-0.088108,-0.084508,-0.039504,-0.067857,-0.053005,-0.080908,-0.124561,-0.005752,-0.089008,-0.053455,...,-0.067407,-0.017002,-0.103409,-0.083608,-0.025553,-0.054805,0.020801,-0.003501,-0.029153,0
4,0.136010,0.146811,0.200815,0.159862,0.163912,0.123859,0.130159,0.105407,0.109008,0.039702,...,0.091456,0.050503,0.057704,0.041502,0.019901,0.015850,0.043302,0.035652,0.047353,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,-0.056606,-0.027803,-0.049405,-0.039504,-0.010252,0.011350,0.004599,0.000099,-0.017452,-0.039504,...,0.022601,-0.032304,-0.155614,-0.108810,-0.101159,-0.018353,-0.080458,-0.070107,-0.024203,1
492,-0.022241,-0.010932,-0.014551,-0.015003,-0.006860,-0.017718,-0.023146,-0.025408,-0.015908,-0.029480,...,-0.015456,0.015307,0.024807,0.003545,-0.018170,-0.016813,0.002188,-0.003693,-0.000979,4
493,0.021188,0.054665,0.073213,0.074118,0.048332,0.034760,0.027974,0.044713,0.057832,0.051951,...,0.275885,0.258242,0.268647,0.303029,0.313886,0.315243,0.309815,0.301219,0.291267,3
494,0.084976,0.079547,0.075928,0.056022,0.067785,0.084071,0.134286,0.123881,0.118453,0.083618,...,0.103976,0.122977,0.118453,0.092214,0.057832,0.060546,0.058737,0.056475,0.035665,0


In [36]:
x_sub_set=subject_481_tst_df.drop([3000],axis=1)

In [ ]:
x_sub_set

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.026201,0.021701,0.037902,0.021701,0.022601,-0.004401,0.019901,0.011800,-0.037704,0.031151,...,0.049153,0.068504,0.052753,0.062654,0.027101,-0.001251,-0.031404,0.004599,-0.030053,-0.027803
1,-0.102959,-0.104309,-0.099359,-0.129061,-0.121861,-0.101159,-0.045355,-0.039954,-0.031404,-0.017902,...,0.008200,0.038802,0.087406,0.138260,0.129709,0.124309,0.086506,0.072105,0.023501,-0.022853
2,-0.122673,-0.129459,-0.115887,-0.080600,-0.076529,-0.056171,-0.057076,-0.059790,-0.055719,-0.036718,...,0.084071,0.084071,0.064618,0.064166,0.050594,0.028427,0.027974,0.044713,0.058284,0.060999
3,-0.088108,-0.084508,-0.039504,-0.067857,-0.053005,-0.080908,-0.124561,-0.005752,-0.089008,-0.053455,...,-0.087208,-0.067407,-0.017002,-0.103409,-0.083608,-0.025553,-0.054805,0.020801,-0.003501,-0.029153
4,0.136010,0.146811,0.200815,0.159862,0.163912,0.123859,0.130159,0.105407,0.109008,0.039702,...,0.070305,0.091456,0.050503,0.057704,0.041502,0.019901,0.015850,0.043302,0.035652,0.047353
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,-0.056606,-0.027803,-0.049405,-0.039504,-0.010252,0.011350,0.004599,0.000099,-0.017452,-0.039504,...,-0.059756,0.022601,-0.032304,-0.155614,-0.108810,-0.101159,-0.018353,-0.080458,-0.070107,-0.024203
492,-0.022241,-0.010932,-0.014551,-0.015003,-0.006860,-0.017718,-0.023146,-0.025408,-0.015908,-0.029480,...,-0.045766,-0.015456,0.015307,0.024807,0.003545,-0.018170,-0.016813,0.002188,-0.003693,-0.000979
493,0.021188,0.054665,0.073213,0.074118,0.048332,0.034760,0.027974,0.044713,0.057832,0.051951,...,0.320672,0.275885,0.258242,0.268647,0.303029,0.313886,0.315243,0.309815,0.301219,0.291267
494,0.084976,0.079547,0.075928,0.056022,0.067785,0.084071,0.134286,0.123881,0.118453,0.083618,...,0.095381,0.103976,0.122977,0.118453,0.092214,0.057832,0.060546,0.058737,0.056475,0.035665


In [37]:
x_sub_set_rshp=np.array(x_sub_set).reshape(496,3000,1)

In [ ]:
x_sub_set_rshp.shape

(496, 3000, 1)

In [38]:
y_sub_dt=subject_481_tst_df[3000]

In [ ]:
y_sub_dt

0      4
1      2
2      2
3      0
4      1
      ..
491    1
492    4
493    3
494    0
495    2
Name: 3000, Length: 496, dtype: int64

In [39]:
y_sub_dt_=np_utils.to_categorical(y_sub_dt)

In [ ]:
y_sub_dt_

array([[0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [14]:
pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-yp4olimg
  Running command git clone --filter=blob:none --quiet https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-yp4olimg
  Resolved https://www.github.com/keras-team/keras-contrib.git to commit 3fc5ef709e061416f4bc8a92ca3750c824b5d2b0
  Preparing metadata (setup.py) ... done
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-py3-none-any.whl size=101055 sha256=b7f3ce4def2117eb7bb23fde774342e309ed4739e4208a8c5955638a12199a9e
  Stored in directory: /tmp/pip-ephem-wheel-cache-qdd3zlay/wheels/74/d5/f7/0245af7ac33d5b0c2e095688649916e4bf9a8d6b3362a849f5
Successfully built keras-contrib


In [15]:
from keras_contrib.layers import CRF

In [16]:
from keras import optimizers, losses, activations, models
from keras.layers import Dense, Input, Dropout, Convolution1D, MaxPool1D, GlobalMaxPool1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D, TimeDistributed, Bidirectional, LSTM
from keras_contrib.layers import CRF

WINDOW_SIZE = 100

In [17]:
nclass = 5

In [ ]:
#n_classes=5
def model_b(n_classes=5, use_sub_layer=False, use_rnn=True, verbose=False):
    inputLayer = Input(shape=(3000, 1), name='inLayer')
    convFine = Conv1D(filters=64, kernel_size=int(Fs/2), strides=int(Fs/16), padding='same', activation='relu', name='fConv1')(inputLayer)
    convFine = MaxPool1D(pool_size=8, strides=8, name='fMaxP1')(convFine)
    convFine = Dropout(rate=0.5, name='fDrop1')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv2')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv3')(convFine)
    convFine = Conv1D(filters=128, kernel_size=8, padding='same', activation='relu', name='fConv4')(convFine)
    convFine = MaxPool1D(pool_size=4, strides=4, name='fMaxP2')(convFine)
    fineShape = convFine.get_shape()
    convFine = Flatten(name='fFlat1')(convFine)

    # network to learn coarse features
    convCoarse = Conv1D(filters=32, kernel_size=Fs*4, strides=int(Fs/2), padding='same', activation='relu', name='cConv1')(inputLayer)
    convCoarse = MaxPool1D(pool_size=4, strides=4, name='cMaxP1')(convCoarse)
    convCoarse = Dropout(rate=0.5, name='cDrop1')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv2')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv3')(convCoarse)
    convCoarse = Conv1D(filters=128, kernel_size=6, padding='same', activation='relu', name='cConv4')(convCoarse)
    convCoarse = MaxPool1D(pool_size=2, strides=2, name='cMaxP2')(convCoarse)
    coarseShape = convCoarse.get_shape()
    convCoarse = Flatten(name='cFlat1')(convCoarse)

    # concatenate coarse and fine cnns
    mergeLayer = concatenate([convFine, convCoarse], name='merge_1')
    outLayer = Dropout(rate=0.5, name='mDrop1')(mergeLayer)

    outLayer = Reshape((1, outLayer.get_shape()[1]), name='reshape1')(outLayer)
    outLayer = LSTM(64, return_sequences=True)(outLayer)
    outLayer = LSTM(64, return_sequences=False)(outLayer)

    # Classify
    outLayer = Dense(n_classes, activation='softmax', name='outLayer')(outLayer)
    model = Model(inputLayer, outLayer)
    optimizer = Adam(lr=1e-4)
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['acc'])
    #model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    if verbose:
        model.summary()
    return model

Checking for individual model accuracies wrt to the test set

In [ ]:
loaded_model_shuf1=model_b(verbose=VBS)
loaded_model_shuf2=model_b(verbose=VBS)
loaded_model_shuf3=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
#dt_shuf1
checkpoint_path_load_model_30_epochs_shuf1 = "/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf1_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf1)
#latest_30_epochs_shuf1_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf1_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf1)

/content/gdrive/My Drive/physionet/dt_shuf1_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf1_30_epochs.ckpt


In [ ]:
#dt_shuf2
checkpoint_path_load_model_30_epochs_shuf2 = "/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf2_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf2)
#latest_300_epochs_shuf2_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf2_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf2)

/content/gdrive/My Drive/physionet/dt_shuf2_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf2_30_epochs.ckpt


In [ ]:
#dt_shuf3
checkpoint_path_load_model_30_epochs_shuf3 = "/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt"
checkpoint_load_model_30_epochs_shuf3_dir = os.path.dirname(checkpoint_path_load_model_30_epochs_shuf3)
#latest_300_epochs_shuf3_s828=tensorflow.train.latest_checkpoint(checkpoint_load_model_300_epochs_shuf3_s828_dir)
print(checkpoint_path_load_model_30_epochs_shuf3)

/content/gdrive/My Drive/physionet/dt_shuf3_phy/sub_482_75th_sub_with_50_sub_tr_dt/weights-phy_shuf3_30_epochs.ckpt


In [ ]:
loaded_model_shuf1.load_weights(checkpoint_path_load_model_30_epochs_shuf1)
loaded_model_shuf2.load_weights(checkpoint_path_load_model_30_epochs_shuf2)
loaded_model_shuf3.load_weights(checkpoint_path_load_model_30_epochs_shuf3)

In [ ]:
optimizer = Adam(lr=1e-4)

In [ ]:
loaded_model_shuf1.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf2.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model_shuf3.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
loaded_model_shuf1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
loaded_model_shuf2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
loaded_model_shuf3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                            

In [ ]:
preds_m1 = loaded_model_shuf1.predict(x_sub_set_rshp)
preds_m1

array([[9.09095816e-03, 8.80166739e-02, 5.98391667e-02, 9.15755081e-05,
        8.42961669e-01],
       [3.45676090e-04, 1.21684205e-02, 9.81667399e-01, 2.56890291e-03,
        3.24961427e-03],
       [4.49957479e-05, 1.36625674e-03, 9.94424284e-01, 1.82588631e-03,
        2.33855681e-03],
       ...,
       [2.96721904e-04, 1.19348802e-03, 3.06014955e-01, 6.92255020e-01,
        2.39852801e-04],
       [9.95429397e-01, 3.52478633e-03, 3.23247048e-04, 5.78260297e-05,
        6.64749241e-04],
       [6.42487430e-05, 3.93863395e-03, 9.94493246e-01, 6.78736600e-04,
        8.25152500e-04]], dtype=float32)

In [ ]:
preds_m2 = loaded_model_shuf2.predict(x_sub_set_rshp)
preds_m2

array([[2.16020159e-02, 2.86050171e-01, 5.12999222e-02, 1.08726044e-05,
        6.41037047e-01],
       [3.77965788e-03, 1.37492791e-01, 8.54426026e-01, 4.26202139e-04,
        3.87534476e-03],
       [5.51248995e-05, 1.57884229e-03, 9.94818032e-01, 2.46819435e-03,
        1.07976061e-03],
       ...,
       [1.11020345e-04, 5.63990965e-04, 4.69086885e-01, 5.29625058e-01,
        6.13089243e-04],
       [9.91531432e-01, 7.38599990e-03, 1.89782411e-04, 1.52278717e-05,
        8.77673505e-04],
       [1.14108720e-04, 4.96073114e-03, 9.93996382e-01, 3.13592143e-04,
        6.15215045e-04]], dtype=float32)

In [ ]:
preds_m3 = loaded_model_shuf3.predict(x_sub_set_rshp)
preds_m3

array([[7.60465721e-03, 1.06796898e-01, 7.26375729e-02, 2.31922841e-05,
        8.12937677e-01],
       [8.96413811e-04, 3.39324027e-02, 9.61639702e-01, 5.01133909e-04,
        3.03026801e-03],
       [2.83590016e-05, 6.27823989e-04, 9.96076405e-01, 2.52715731e-03,
        7.40294170e-04],
       ...,
       [1.11981215e-04, 3.02836561e-04, 2.32980281e-01, 7.66378641e-01,
        2.26210439e-04],
       [9.92050827e-01, 4.26374469e-03, 2.50250159e-04, 3.45155713e-05,
        3.40072392e-03],
       [1.69225212e-04, 9.94790159e-03, 9.88698781e-01, 3.66225198e-04,
        8.17831722e-04]], dtype=float32)

In [ ]:
preds_m1_fin=np.argmax(preds_m1,axis=1)
preds_m2_fin=np.argmax(preds_m2,axis=1)
preds_m3_fin=np.argmax(preds_m3,axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m1 = confusion_matrix(y_sub_dt, preds_m1_fin)
matrix_m1.diagonal()/matrix_m1.sum(axis=1)

array([0.88095238, 0.31372549, 0.94152047, 0.7826087 , 0.9       ])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m2 = confusion_matrix(y_sub_dt, preds_m2_fin)
matrix_m2.diagonal()/matrix_m2.sum(axis=1)

array([0.86309524, 0.43137255, 0.96491228, 0.65217391, 0.86666667])

In [ ]:
#from sklearn.metrics import confusion_matrix
matrix_m3 = confusion_matrix(y_sub_dt, preds_m3_fin)
matrix_m3.diagonal()/matrix_m3.sum(axis=1)

array([0.875     , 0.33333333, 0.93567251, 0.95652174, 0.91666667])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m1 = f1_score(y_sub_dt, preds_m1_fin, average="macro")
prec_m1 = precision_score(y_sub_dt, preds_m1_fin, average="macro")
recall_m1 = recall_score(y_sub_dt, preds_m1_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m1))
    print("Recall: {}".format(recall_m1))
    print("F1 score: {}".format(f1_m1))

    report_m1 = classification_report(y_sub_dt, preds_m1_fin, digits=6)
    print(report_m1)

Precision: 0.7885203435804702
Recall: 0.763761406927378
F1 score: 0.7701183434433174
              precision    recall  f1-score   support

           0   0.936709  0.880952  0.907975       168
           1   0.500000  0.313725  0.385542        51
           2   0.805000  0.941520  0.867925       171
           3   0.857143  0.782609  0.818182        46
           4   0.843750  0.900000  0.870968        60

    accuracy                       0.836694       496
   macro avg   0.788520  0.763761  0.770118       496
weighted avg   0.827774  0.836694  0.827645       496



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m2 = f1_score(y_sub_dt, preds_m2_fin, average="macro")
prec_m2 = precision_score(y_sub_dt, preds_m2_fin, average="macro")
recall_m2 = recall_score(y_sub_dt, preds_m2_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m2))
    print("Recall: {}".format(recall_m2))
    print("F1 score: {}".format(f1_m2))

    report_m2 = classification_report(y_sub_dt, preds_m2_fin, digits=6)
    print(report_m2)

Precision: 0.8152322248773235
Recall: 0.7556441295053491
F1 score: 0.7769648932029863
              precision    recall  f1-score   support

           0   0.960265  0.863095  0.909091       168
           1   0.458333  0.431373  0.444444        51
           2   0.793269  0.964912  0.870712       171
           3   0.967742  0.652174  0.779221        46
           4   0.896552  0.866667  0.881356        60

    accuracy                       0.834677       496
   macro avg   0.815232  0.755644  0.776965       496
weighted avg   0.844068  0.834677  0.832684       496



In [ ]:
#y_sub_dt
#from sklearn.metrics import precision_score, recall_score
f1_m3 = f1_score(y_sub_dt, preds_m3_fin, average="macro")
prec_m3 = precision_score(y_sub_dt, preds_m3_fin, average="macro")
recall_m3 = recall_score(y_sub_dt, preds_m3_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m3))
    print("Recall: {}".format(recall_m3))
    print("F1 score: {}".format(f1_m3))

    report_m3 = classification_report(y_sub_dt, preds_m3_fin, digits=6)
    print(report_m3)

Precision: 0.8005161877950469
Recall: 0.8034388507500635
F1 score: 0.794623245698981
              precision    recall  f1-score   support

           0   0.942308  0.875000  0.907407       168
           1   0.548387  0.333333  0.414634        51
           2   0.837696  0.935673  0.883978       171
           3   0.814815  0.956522  0.880000        46
           4   0.859375  0.916667  0.887097        60

    accuracy                       0.852823       496
   macro avg   0.800516  0.803439  0.794623       496
weighted avg   0.843882  0.852823  0.843663       496



In [ ]:
check_pt_path="/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt"

In [ ]:
check_pt_path="/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt"

In [ ]:
#print(latest_teacher_30_epochs)
print(check_pt_path)

/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt


In [ ]:
teacher_30_epochs=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_30_epochs.load_weights(check_pt_path)

In [ ]:
optimizer=Adam(lr=1e-4)

In [ ]:
accuracy_teacher_model_30_epochs_wts_loaded = teacher_30_epochs.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 12s 9ms/step - loss: 0.7530 - acc: 0.8306


In [ ]:
preds_teacher = teacher_30_epochs.predict(x_sub_set_rshp)

In [ ]:
preds_teacher

array([[1.93090327e-05, 1.46171649e-03, 7.06946477e-04, 2.54745089e-07,
        9.97811854e-01],
       [4.95352378e-06, 3.74415977e-05, 9.99899626e-01, 3.20490835e-05,
        2.58152795e-05],
       [1.70547719e-06, 3.21340985e-06, 9.99937177e-01, 3.92491711e-05,
        1.87397491e-05],
       ...,
       [7.94460866e-05, 4.19738062e-05, 8.65633905e-01, 1.34232506e-01,
        1.22090623e-05],
       [9.98519480e-01, 1.19813754e-04, 1.49490133e-05, 2.39288602e-05,
        1.32171356e-03],
       [1.17589877e-06, 1.51540971e-05, 9.99966502e-01, 8.91519539e-06,
        8.27779513e-06]], dtype=float32)

In [ ]:
preds_teacher_fin=np.argmax(preds_teacher,axis=1)

In [ ]:
preds_teacher_fin

array([4, 2, 2, 0, 0, 2, 3, 0, 1, 2, 0, 2, 0, 2, 1, 1, 2, 2, 0, 0, 4, 1,
       1, 2, 0, 0, 3, 2, 2, 4, 3, 2, 0, 2, 0, 1, 0, 1, 0, 2, 2, 0, 0, 0,
       0, 2, 2, 0, 0, 0, 3, 2, 2, 0, 0, 0, 2, 0, 0, 3, 3, 0, 3, 2, 0, 2,
       4, 2, 0, 2, 2, 2, 2, 0, 1, 0, 0, 3, 0, 2, 0, 4, 2, 1, 2, 2, 0, 1,
       2, 2, 0, 0, 2, 2, 2, 4, 1, 2, 4, 2, 3, 0, 1, 4, 2, 0, 3, 2, 0, 2,
       0, 4, 2, 2, 1, 2, 2, 2, 4, 4, 4, 2, 2, 0, 2, 1, 2, 0, 2, 4, 2, 4,
       2, 2, 2, 1, 3, 0, 3, 4, 4, 3, 2, 2, 0, 1, 2, 0, 2, 2, 4, 2, 2, 2,
       0, 2, 0, 4, 0, 2, 2, 4, 2, 0, 0, 0, 4, 0, 2, 0, 2, 0, 0, 0, 2, 2,
       4, 2, 0, 0, 4, 0, 0, 0, 2, 2, 2, 2, 2, 0, 3, 2, 0, 2, 2, 1, 2, 0,
       2, 0, 2, 2, 4, 2, 0, 0, 4, 2, 2, 0, 4, 2, 0, 2, 0, 2, 2, 2, 4, 2,
       0, 0, 0, 3, 0, 0, 2, 4, 0, 2, 0, 0, 4, 1, 1, 1, 2, 2, 0, 0, 2, 1,
       1, 0, 2, 4, 2, 3, 4, 0, 4, 4, 2, 4, 2, 0, 2, 2, 0, 3, 4, 4, 3, 0,
       0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 3, 0, 2, 2, 2, 1, 0, 0, 0, 0,
       2, 2, 4, 4, 0, 2, 0, 4, 2, 1, 0, 2, 2, 2, 4,

In [ ]:
#accuracy after incrementally training and predicting on x_train_sub[0:500](hold-out set) using loaded_model_shuf1_inc
from sklearn.metrics import accuracy_score
accuracy_score(y_sub_dt, preds_teacher_fin)

0.8306451612903226

In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, preds_teacher_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.86309524, 0.31372549, 0.97076023, 0.65217391, 0.91666667])

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_cnn = f1_score(y_sub_dt, preds_teacher_fin, average="macro")
prec_cnn = precision_score(y_sub_dt, preds_teacher_fin, average="macro")
recall_cnn = recall_score(y_sub_dt, preds_teacher_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_cnn))
    print("Recall: {}".format(recall_cnn))
    print("F1 score: {}".format(f1_cnn))

    report = classification_report(y_sub_dt, preds_teacher_fin, digits=6)
    print(report)

Precision: 0.7951523441130006
Recall: 0.743284308383918
F1 score: 0.7593360748599156
              precision    recall  f1-score   support

           0   0.947712  0.863095  0.903427       168
           1   0.444444  0.313725  0.367816        51
           2   0.786730  0.970760  0.869110       171
           3   0.937500  0.652174  0.769231        46
           4   0.859375  0.916667  0.887097        60

    accuracy                       0.830645       496
   macro avg   0.795152  0.743284  0.759336       496
weighted avg   0.828832  0.830645  0.822102       496



In [18]:
from keras.layers import Input, Dense, LSTM, MaxPooling1D, Conv1D
from keras.models import Model

In [19]:
#student model
def get_model_1d_cnn():
    nclass = 5
    inp = Input(shape=(3000, 1))
    img_1 = Conv1D(4, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(inp)
    #img_1 = Conv1D(128, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.2)(img_1)
    img_1 = Conv1D(4, kernel_size=5, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(256, kernel_size=7, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = Conv1D(64, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = SpatialDropout1D(rate=0.01)(img_1)
    img_1 = Conv1D(10, kernel_size=4, activation=activations.relu, strides=1, padding="valid")(img_1)
    img_1 = MaxPooling1D(pool_size=2)(img_1)
    #img_1 = Conv1D(64, kernel_size=6, activation=activations.relu, strides=1, padding="valid")(img_1)
    #img_1 = MaxPooling1D(pool_size=2)(img_1)
    img_1 = Flatten()(img_1)


    dense_1 = Dropout(rate=0.5)(img_1)
    #dense_1 = Dropout(rate=0.05)(Dense(64, activation=activations.relu, name="dense_2")(dense_1))
    dense_1 = Dense(nclass, activation=activations.softmax)(dense_1)

    model = Model(inputs=inp, outputs=dense_1)
    opt = Adam(0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['acc'])
    model.summary()
    return model

In [ ]:
m=get_model_1d_cnn()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170   

In [ ]:
m.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
hist_19_cnn = m.fit(x_transfer_set_rshp, y_transfer_, epochs=30)

Epoch 1/30
3648/3648 [==============================] - 30s 5ms/step - loss: 1.1210 - accuracy: 0.5474
Epoch 2/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.8103 - accuracy: 0.6878
Epoch 3/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.7550 - accuracy: 0.7081
Epoch 4/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.7294 - accuracy: 0.7143
Epoch 5/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.7075 - accuracy: 0.7192
Epoch 6/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.6859 - accuracy: 0.7277
Epoch 7/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.6649 - accuracy: 0.7341
Epoch 8/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.6443 - accuracy: 0.7418
Epoch 9/30
3648/3648 [==============================] - 19s 5ms/step - loss: 0.6236 - accuracy: 0.7490
Epoch 10/30
3648/3648 [==============================] - 19s 5ms/step - l

In [ ]:
accuracy_1d_cnn = m.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 0s 4ms/step - loss: 0.6009 - accuracy: 0.7762


In [ ]:
preds_m = m.predict(x_sub_set_rshp)

In [ ]:
preds_m_fin=np.argmax(preds_m,axis=1)

In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_m = f1_score(y_sub_dt, preds_m_fin, average="macro")
prec_m = precision_score(y_sub_dt, preds_m_fin, average="macro")
recall_m = recall_score(y_sub_dt, preds_m_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_m))
    print("Recall: {}".format(recall_m))
    print("F1 score: {}".format(f1_m))

    report = classification_report(y_sub_dt, preds_m_fin, digits=6)
    print(report)

Precision: 0.7524285981788733
Recall: 0.6346764609734054
F1 score: 0.6601169298539233
              precision    recall  f1-score   support

           0   0.881356  0.928571  0.904348       168
           1   0.444444  0.156863  0.231884        51
           2   0.695279  0.947368  0.801980       171
           3   0.837838  0.673913  0.746988        46
           4   0.903226  0.466667  0.615385        60

    accuracy                       0.776210       496
   macro avg   0.752429  0.634676  0.660117       496
weighted avg   0.770890  0.776210  0.750362       496



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_m = confusion_matrix(y_sub_dt, preds_m_fin)
matrix_m.diagonal()/matrix_m.sum(axis=1)

array([0.92857143, 0.15686275, 0.94736842, 0.67391304, 0.46666667])

In [ ]:
cm_m = matrix_m.astype('float') / matrix_m.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_m.diagonal()

array([0.92857143, 0.15686275, 0.94736842, 0.67391304, 0.46666667])

# light weight model trained on transfer set + subject specific train set and testing on subject specific test set

In [20]:
subject_481_tr_df=pd.read_csv('/content/gdrive/My Drive/sub_481/x_train_sub.csv',header=None)

In [21]:
subject_481_tr_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.008521,-0.021337,-0.023599,-0.011384,-0.005051,0.006259,0.029784,0.019831,0.020283,0.036570,...,-0.061147,-0.065219,-0.065671,-0.072457,-0.076529,-0.066124,-0.045766,-0.029027,-0.038528,4
1,0.036570,0.025712,0.022093,0.026617,0.027974,-0.013194,-0.069290,-0.087838,-0.075171,-0.038075,...,0.035665,-0.002336,-0.024503,-0.052099,-0.042599,-0.019527,-0.012289,0.003545,0.006259,1
2,0.029801,0.029351,0.076605,-0.003951,0.022151,0.047353,0.165262,-0.051205,0.033402,0.034752,...,-0.077307,-0.083158,-0.055256,-0.005752,0.001449,-0.024203,-0.005752,-0.037704,-0.065156,0
3,0.047427,0.085428,0.082714,0.083166,0.094476,0.084523,0.072309,0.045165,0.022998,0.009878,...,-0.087838,-0.101863,-0.108648,-0.117244,-0.102315,-0.087386,-0.079695,-0.070195,-0.051647,4
4,0.416383,0.437534,0.459136,0.460486,0.450135,0.424033,0.438884,0.481638,0.511340,0.514041,...,-0.035454,-0.004401,-0.003501,-0.009352,0.010450,0.022151,0.038802,0.040152,0.070305,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,0.003997,0.004450,0.019831,0.028427,0.027974,0.005354,-0.015456,-0.011384,0.017117,0.010783,...,0.001735,-0.032194,-0.073814,-0.102315,-0.091005,-0.076076,-0.055719,-0.005955,0.046522,4
1976,-0.057506,-0.007102,0.013150,0.029351,0.032502,0.050503,0.029351,0.045103,0.082906,0.074355,...,0.189114,0.132410,0.136010,0.131060,0.136010,0.144561,0.173813,0.172463,0.131510,2
1977,-0.043052,-0.035361,-0.011384,-0.004146,-0.014551,-0.032194,-0.029027,-0.030837,-0.040790,-0.036266,...,0.037474,0.008069,-0.022694,-0.051647,-0.032646,-0.035361,-0.037623,-0.055719,-0.073814,2
1978,-0.006202,-0.000351,0.003699,-0.012502,-0.079557,-0.057506,0.011350,-0.030053,-0.048955,0.048253,...,0.025751,0.022151,0.038802,0.039252,0.010900,0.013600,-0.029603,-0.034104,-0.066056,0


In [22]:
transfer_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116713,0.032558,0.033944,0.046413,0.056111,0.060267,0.066271,0.077816,0.058882,0.078278,0.051493,...,0.024707,0.025631,0.015933,0.044565,-0.000231,0.018242,-0.028402,0.021475,0.028402,2
116714,0.068695,0.059483,0.051071,0.044662,0.037051,0.041057,0.040656,0.046264,0.039054,0.045063,...,0.035850,0.031444,0.026637,0.023433,0.027839,0.023833,0.018626,0.005007,0.007010,4
116715,0.013392,0.030717,0.029805,0.016127,0.026613,0.024334,0.019775,0.038012,0.027069,0.031629,...,0.013392,0.018863,0.034820,0.034364,0.042115,0.045306,0.041659,0.032996,0.024334,4
116716,-0.057457,-0.065629,-0.101198,-0.114176,-0.141093,-0.184354,-0.222807,-0.230017,-0.242514,-0.231459,...,-0.085817,-0.068993,-0.058899,-0.043999,-0.058419,-0.065629,-0.070916,-0.068513,-0.090143,2


In [23]:
transfer_df_plus_sub_tr = transfer_df.append(subject_481_tr_df)

In [24]:
transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,3000
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125,4
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467,3
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651,2
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754,2
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,0.003997,0.004450,0.019831,0.028427,0.027974,0.005354,-0.015456,-0.011384,0.017117,0.010783,...,0.001735,-0.032194,-0.073814,-0.102315,-0.091005,-0.076076,-0.055719,-0.005955,0.046522,4
1976,-0.057506,-0.007102,0.013150,0.029351,0.032502,0.050503,0.029351,0.045103,0.082906,0.074355,...,0.189114,0.132410,0.136010,0.131060,0.136010,0.144561,0.173813,0.172463,0.131510,2
1977,-0.043052,-0.035361,-0.011384,-0.004146,-0.014551,-0.032194,-0.029027,-0.030837,-0.040790,-0.036266,...,0.037474,0.008069,-0.022694,-0.051647,-0.032646,-0.035361,-0.037623,-0.055719,-0.073814,2
1978,-0.006202,-0.000351,0.003699,-0.012502,-0.079557,-0.057506,0.011350,-0.030053,-0.048955,0.048253,...,0.025751,0.022151,0.038802,0.039252,0.010900,0.013600,-0.029603,-0.034104,-0.066056,0


In [25]:
x_transfer_df_plus_sub_tr=transfer_df_plus_sub_tr.drop([3000],axis=1)

In [26]:
x_transfer_df_plus_sub_tr

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
0,0.068014,0.082915,0.077628,0.074744,0.060804,0.063688,0.050710,0.029561,0.010815,0.008412,...,-0.007450,-0.010334,-0.016102,-0.034368,-0.055998,-0.074263,-0.071860,-0.077147,-0.084838,-0.090125
1,0.104729,0.054513,-0.020131,-0.064918,-0.122825,-0.127801,-0.107443,-0.086633,-0.051799,-0.033251,...,-0.102467,-0.123729,-0.120563,-0.129158,-0.149968,-0.173945,-0.187064,-0.171231,-0.149516,-0.121467
2,0.002281,-0.031309,-0.007672,-0.020527,-0.003110,-0.011404,0.006013,-0.029236,-0.025089,-0.042091,...,0.054532,0.088122,0.057850,0.027992,0.038359,0.040847,0.053703,0.072779,0.055362,0.029651
3,-0.025235,-0.037732,-0.033406,-0.031964,-0.034848,-0.041097,-0.031484,-0.025716,-0.019948,-0.030522,...,-0.012738,-0.015141,-0.013699,-0.018025,-0.020909,-0.021390,-0.002644,-0.020428,-0.025235,-0.024754
4,0.097914,-0.100520,-0.044574,-0.079540,-0.071235,-0.029713,-0.041951,0.000445,-0.031024,0.172217,...,0.264003,0.167409,0.220732,0.191011,0.119767,0.165661,0.270996,0.200190,0.246520,0.166535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,0.003997,0.004450,0.019831,0.028427,0.027974,0.005354,-0.015456,-0.011384,0.017117,0.010783,...,0.041546,0.001735,-0.032194,-0.073814,-0.102315,-0.091005,-0.076076,-0.055719,-0.005955,0.046522
1976,-0.057506,-0.007102,0.013150,0.029351,0.032502,0.050503,0.029351,0.045103,0.082906,0.074355,...,0.200365,0.189114,0.132410,0.136010,0.131060,0.136010,0.144561,0.173813,0.172463,0.131510
1977,-0.043052,-0.035361,-0.011384,-0.004146,-0.014551,-0.032194,-0.029027,-0.030837,-0.040790,-0.036266,...,0.017117,0.037474,0.008069,-0.022694,-0.051647,-0.032646,-0.035361,-0.037623,-0.055719,-0.073814
1978,-0.006202,-0.000351,0.003699,-0.012502,-0.079557,-0.057506,0.011350,-0.030053,-0.048955,0.048253,...,0.033852,0.025751,0.022151,0.038802,0.039252,0.010900,0.013600,-0.029603,-0.034104,-0.066056


In [27]:
y_transfer_df_plus_sub_tr = transfer_df_plus_sub_tr[3000]

In [28]:
y_transfer_df_plus_sub_tr

0       4
1       3
2       2
3       2
4       0
       ..
1975    4
1976    2
1977    2
1978    0
1979    2
Name: 3000, Length: 118698, dtype: int64

In [29]:
x_transfer_df_plus_sub_tr_rshp=np.array(x_transfer_df_plus_sub_tr).reshape(118698,3000,1)

In [30]:
x_transfer_df_plus_sub_tr_rshp.shape

(118698, 3000, 1)

In [31]:
y_transfer_df_plus_sub_tr_=np_utils.to_categorical(y_transfer_df_plus_sub_tr)

In [32]:
y_transfer_df_plus_sub_tr_

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.]], dtype=float32)

In [67]:
light_wt_model=get_model_1d_cnn()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d_16 (Conv1D)          (None, 2995, 4)           28        
                                                                 
 max_pooling1d_16 (MaxPoolin  (None, 1497, 4)          0         
 g1D)                                                            
                                                                 
 conv1d_17 (Conv1D)          (None, 1493, 4)           84        
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 746, 4)           0         
 g1D)                                                            
                                                                 
 conv1d_18 (Conv1D)          (None, 743, 10)           170 

In [68]:
hist = light_wt_model.fit(x_transfer_df_plus_sub_tr_rshp, y_transfer_df_plus_sub_tr_, epochs=30)

Epoch 1/30
3710/3710 [==============================] - 18s 4ms/step - loss: 0.7577 - acc: 0.6933
Epoch 2/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.5902 - acc: 0.7605
Epoch 3/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.5376 - acc: 0.7846
Epoch 4/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.4878 - acc: 0.8060
Epoch 5/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.4551 - acc: 0.8191
Epoch 6/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.4280 - acc: 0.8291
Epoch 7/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.4137 - acc: 0.8349
Epoch 8/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.4011 - acc: 0.8403
Epoch 9/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.3936 - acc: 0.8439
Epoch 10/30
3710/3710 [==============================] - 16s 4ms/step - loss: 0.3847 - acc: 0.8476
Epoch 11/30
3710/37

In [69]:
accuracy = light_wt_model.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 0s 2ms/step - loss: 0.4631 - acc: 0.8165


In [70]:
preds_light_wt_model = light_wt_model.predict(x_sub_set_rshp)

16/16 [==============================] - 0s 2ms/step


In [71]:
preds_light_wt_model_fin=np.argmax(preds_light_wt_model,axis=1)

In [72]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1_light_wt_model = f1_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
prec_light_wt_model = precision_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
recall_light_wt_model = recall_score(y_sub_dt, preds_light_wt_model_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec_light_wt_model))
    print("Recall: {}".format(recall_light_wt_model))
    print("F1 score: {}".format(f1_light_wt_model))

    report = classification_report(y_sub_dt, preds_light_wt_model_fin, digits=6)
    print(report)

Precision: 0.7721856607811663
Recall: 0.7048400840122472
F1 score: 0.723896107094987
              precision    recall  f1-score   support

           0   0.876404  0.928571  0.901734       168
           1   0.375000  0.176471  0.240000        51
           2   0.776190  0.953216  0.855643       171
           3   0.857143  0.782609  0.818182        46
           4   0.976190  0.683333  0.803922        60

    accuracy                       0.816532       496
   macro avg   0.772186  0.704840  0.723896       496
weighted avg   0.800584  0.816532  0.798222       496



In [73]:
from sklearn.metrics import confusion_matrix
matrix_light_wt_model = confusion_matrix(y_sub_dt, preds_light_wt_model_fin)
matrix_light_wt_model.diagonal()/matrix_light_wt_model.sum(axis=1)

array([0.92857143, 0.17647059, 0.95321637, 0.7826087 , 0.68333333])

In [ ]:
import keras

In [ ]:
teacher_model=model_b(verbose=VBS)

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inLayer (InputLayer)           [(None, 3000, 1)]    0           []                               
                                                                                                  
 fConv1 (Conv1D)                (None, 500, 64)      3264        ['inLayer[0][0]']                
                                                                                                  
 cConv1 (Conv1D)                (None, 60, 32)       12832       ['inLayer[0][0]']                
                                                                                                  
 fMaxP1 (MaxPooling1D)          (None, 62, 64)       0           ['fConv1[0][0]']                 
                                                                                              

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_checkpt_path='/content/gdrive/My Drive/physionet/ensemble/teacher_wts_30_epochs_50_subs_trial_3.ckpt'
teacher_model.load_weights(teacher_checkpt_path)

In [ ]:
teacher_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
accuracy_teacher_model = teacher_model.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 12s 9ms/step - loss: 0.7296 - accuracy: 0.8246


In [ ]:
preds_teacher = teacher_model.predict(x_sub_set_rshp)

In [ ]:
preds_teacher_fin=np.argmax(preds_teacher,axis=1)

In [ ]:
print(classification_report(y_sub_dt, preds_teacher_fin, digits=6))

              precision    recall  f1-score   support

           0   0.948052  0.869048  0.906832       168
           1   0.437500  0.274510  0.337349        51
           2   0.769585  0.976608  0.860825       171
           3   0.933333  0.608696  0.736842        46
           4   0.857143  0.900000  0.878049        60

    accuracy                       0.824597       496
   macro avg   0.789123  0.725772  0.743979       496
weighted avg   0.821666  0.824597  0.813168       496



In [ ]:
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_sub_dt, preds_teacher_fin)
matrix.diagonal()/matrix.sum(axis=1)

array([0.86904762, 0.2745098 , 0.97660819, 0.60869565, 0.9       ])

In [ ]:
cm = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

In [ ]:
cm

array([[0.86904762, 0.08928571, 0.        , 0.        , 0.04166667],
       [0.1372549 , 0.2745098 , 0.54901961, 0.        , 0.03921569],
       [0.        , 0.01169591, 0.97660819, 0.01169591, 0.        ],
       [0.        , 0.        , 0.39130435, 0.60869565, 0.        ],
       [0.01666667, 0.01666667, 0.06666667, 0.        , 0.9       ]])

In [ ]:
cm.diagonal()

array([0.86904762, 0.2745098 , 0.97660819, 0.60869565, 0.9       ])

In [ ]:
#student_model=get_model_cnn() #issue with unmatched shape
student_model=get_model_1d_cnn()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2995, 4)           28        
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1497, 4)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1493, 4)           84        
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 746, 4)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 743, 10)           170 

In [ ]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super(Distiller, self).__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super(Distiller, self).compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)
            distillation_loss = self.distillation_loss_fn(
                tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                tf.nn.softmax(student_predictions / self.temperature, axis=1),
            )
            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np


In [ ]:
# Initialize and compile distiller
distiller = Distiller(student=student_model, teacher=teacher_model)
distiller.compile(
    optimizer=keras.optimizers.Adam(),
    metrics=[keras.metrics.CategoricalAccuracy()],
    student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),
    distillation_loss_fn=keras.losses.KLDivergence(),
    alpha=0.9,
    temperature=4,
)

# Distill teacher to student
distiller.fit(x_transfer_set_rshp, y_transfer_, epochs=40)

# Evaluate student on test dataset
#distiller.evaluate(x_test, y_test)

Epoch 1/40
3648/3648 [==============================] - 47s 9ms/step - categorical_accuracy: 0.6525 - student_loss: 0.8502 - distillation_loss: 0.0027
Epoch 2/40
3648/3648 [==============================] - 31s 9ms/step - categorical_accuracy: 0.7377 - student_loss: 0.6322 - distillation_loss: 0.0020
Epoch 3/40
3648/3648 [==============================] - 31s 9ms/step - categorical_accuracy: 0.7706 - student_loss: 0.5508 - distillation_loss: 0.0018
Epoch 4/40
3648/3648 [==============================] - 32s 9ms/step - categorical_accuracy: 0.7849 - student_loss: 0.5201 - distillation_loss: 0.0017
Epoch 5/40
3648/3648 [==============================] - 31s 9ms/step - categorical_accuracy: 0.7946 - student_loss: 0.4981 - distillation_loss: 0.0016
Epoch 6/40
3648/3648 [==============================] - 31s 9ms/step - categorical_accuracy: 0.8044 - student_loss: 0.4746 - distillation_loss: 0.0015
Epoch 7/40
3648/3648 [==============================] - 31s 9ms/step - categorical_accuracy: 0

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 0s 4ms/step - categorical_accuracy: 0.8145 - student_loss: 0.6017


In [ ]:
preds = student_model.predict(x_sub_set_rshp)

In [ ]:
preds

array([[3.07926573e-02, 5.21272123e-01, 9.26287323e-02, 1.64365073e-08,
        3.55306476e-01],
       [1.83104737e-06, 9.94674774e-05, 9.97432530e-01, 2.46591610e-03,
        2.79289111e-07],
       [2.25709465e-10, 1.00487305e-04, 9.99378800e-01, 5.20724047e-04,
        6.20267615e-10],
       ...,
       [7.34642299e-07, 2.19179012e-04, 8.79621744e-01, 1.20154880e-01,
        3.41218924e-06],
       [9.49193537e-01, 3.26359831e-02, 1.53466230e-02, 3.20663162e-07,
        2.82356469e-03],
       [2.23695586e-08, 1.74879504e-04, 9.99761164e-01, 6.38009296e-05,
        7.52239444e-08]], dtype=float32)

In [ ]:
preds_fin=np.argmax(preds,axis=1)

In [ ]:
preds_fin

array([1, 2, 2, 0, 0, 2, 2, 0, 4, 2, 0, 2, 0, 2, 0, 0, 2, 2, 0, 0, 4, 0,
       1, 2, 0, 0, 3, 2, 2, 4, 3, 2, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0,
       0, 2, 2, 0, 0, 0, 3, 2, 2, 0, 0, 0, 2, 0, 0, 3, 3, 0, 3, 2, 0, 2,
       4, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 3, 0, 2, 0, 2, 2, 1, 2, 2, 0, 2,
       2, 2, 0, 0, 2, 2, 2, 4, 1, 2, 4, 2, 3, 0, 1, 2, 2, 0, 3, 2, 0, 2,
       0, 4, 2, 2, 1, 2, 2, 2, 0, 2, 4, 2, 2, 0, 2, 4, 2, 0, 2, 4, 2, 4,
       2, 2, 2, 0, 3, 0, 3, 4, 2, 3, 3, 2, 1, 1, 2, 0, 2, 2, 4, 2, 2, 2,
       0, 2, 0, 4, 0, 2, 3, 4, 2, 0, 0, 0, 4, 0, 2, 0, 2, 0, 0, 0, 2, 2,
       4, 2, 0, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 3, 2, 0, 2, 2, 1, 2, 0,
       2, 0, 2, 2, 4, 2, 0, 0, 4, 2, 2, 0, 4, 2, 0, 2, 0, 1, 2, 2, 4, 2,
       0, 0, 0, 3, 0, 0, 2, 4, 0, 2, 0, 0, 2, 0, 0, 1, 2, 2, 0, 1, 2, 1,
       0, 1, 2, 4, 1, 3, 4, 0, 1, 4, 2, 4, 2, 0, 2, 2, 0, 3, 1, 4, 3, 0,
       0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 3, 0, 2, 2, 2, 1, 0, 0, 0, 0,
       2, 2, 4, 4, 0, 2, 0, 4, 2, 2, 0, 2, 2, 2, 4,

In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score

In [ ]:
print(accuracy_score(y_sub_dt, preds_fin))
#print(precision_score(y_sub_dt, preds_fin))
#print(recall_score(y_sub_dt, preds_fin))
#print(f1_score(y_sub_dt, preds_fin))

0.8145161290322581


In [ ]:
#y_sub_dt
from sklearn.metrics import precision_score, recall_score
f1 = f1_score(y_sub_dt, preds_fin, average="macro")
prec = precision_score(y_sub_dt, preds_fin, average="macro")
recall = recall_score(y_sub_dt, preds_fin, average="macro")
if VBS:
    print("Precision: {}".format(prec))
    print("Recall: {}".format(recall))
    print("F1 score: {}".format(f1))

    report = classification_report(y_sub_dt, preds_fin, digits=6)
    print(report)

Precision: 0.7690802677131886
Recall: 0.6996606285534808
F1 score: 0.7192364302544954
              precision    recall  f1-score   support

           0   0.912791  0.934524  0.923529       168
           1   0.478261  0.215686  0.297297        51
           2   0.757009  0.947368  0.841558       171
           3   0.825000  0.717391  0.767442        46
           4   0.872340  0.683333  0.766355        60

    accuracy                       0.814516       496
   macro avg   0.769080  0.699661  0.719236       496
weighted avg   0.801369  0.814516  0.797390       496



In [ ]:
from sklearn.metrics import confusion_matrix
matrix_stu = confusion_matrix(y_sub_dt, preds_fin)
matrix_stu.diagonal()/matrix_stu.sum(axis=1)

array([0.93452381, 0.21568627, 0.94736842, 0.7173913 , 0.68333333])

In [ ]:
cm_stu = matrix_stu.astype('float') / matrix_stu.sum(axis=1)[:, np.newaxis]

In [ ]:
cm_stu.diagonal()

array([0.93452381, 0.21568627, 0.94736842, 0.7173913 , 0.68333333])

In [ ]:
accuracy_distiller = distiller.evaluate(x_sub_set_rshp, y_sub_dt_)

16/16 [==============================] - 0s 3ms/step - categorical_accuracy: 0.8145 - student_loss: 0.6017


In [ ]:
#Distiller's performance (student model's prformance)

#alpha=0.9,
#temperature=4

#30 epochs 85.57
#40 epochs 84.85
#50 epochs 83.88
#60 epochs 83.88

#alpha=0.9,
#temperature=5

#30 epochs 85.18
#40 epochs 85.41
#50 epochs 84.01
#60 epochs